For every trip variable, group by day

In [254]:
import pandas as pd

In [255]:
data_folder = '../data/'
trips = pd.read_csv(data_folder + 'trips/test/loaded_trips/trips_test.csv')

from modules.DataPreparation import DataPreparation
dp = DataPreparation()
trips = dp.transform_to_datetime(trips, ['starttime', 'stoptime'])
trips = dp.transform_to_time_series(trips, 'starttime')
trips

,Index,trip_id,tripduration,start_station_name,starttime,end_station_name,stoptime,birth_year,initial_station_latitude,initial_station_longitude,final_station_latitude,final_station_longitude,per_day,hour,week_day,weekend,holiday
date,,,,,,,,,,,,,,,,,
2020-02-16,1553109,6517937BikeSampa,724,6 - Rua Manduri,2020-02-16 23:52:18,223 -Fradique Coutinho,2020-02-17 00:04:22,2050.0,NaN,NaN,NaN,NaN,2020-02-16,23,6,True,False
2020-02-16,1553110,6517938BikeSampa,4332,204- Consolação,2020-02-16 23:52:49,204- Consolação,2020-02-17 01:05:01,1900.0,NaN,NaN,NaN,NaN,2020-02-16,23,6,True,False
2020-02-16,1553111,6517939BikeSampa,2168,95 - Metrô Brigadeiro,2020-02-16 01:53:35,153 - Monsenhor Passaláqua,2020-02-17 00:29:43,2001.0,NaN,NaN,NaN,NaN,2020-02-16,1,6,True,False
2020-02-18,1567892,6540675BikeSampa,787,12 - Largo de Batata II,2020-02-18 09:04:21,17 - Rua José Gonçalves de Oliveira,2020-02-18 09:17:28,1994.0,NaN,NaN,NaN,NaN,2020-02-18,9,1,False,True
2020-02-18,1567893,6540678BikeSampa,628,347 - Metrô Brooklin,2020-02-18 09:04:34,345 - R. Henrique FIsher,2020-02-18 09:15:02,1988.0,NaN,NaN,NaN,NaN,2020-02-18,9,1,False,True
2020-02-19,1584654,6564502BikeSampa,920,154 - R. Condessa de São Joaquim,2020-02-19 17:09:46,167 - Praca Roosevelt,2020-02-19 17:25:06,NaN,NaN,NaN,NaN,NaN,2020-02-19,17,2,False,False


### What variables are relevant to calculate daily average and how to do it

Columns:

- tripduration: get daily trip duration average 
- start_station_name: not relevant for now
- starttime: use column already separate
- end_station_name: not relevant for now
- stoptime: not relevant for now
- birth_year: calculate age and then calculate average
- initial_station_latitude: not relevant for now
- initial_station_longitude: not relevant for now
- final_station_latitude: not relevant for now
- final_station_longitude: not relevant for now
- per_day: already in the time series index
- hour: calculate daily average for now
    - (later) calculate daily trips, afternoon
- week_day: It will be the same to the same day. So get one value or average - same
- weekend: It will be the same to the same day. So get one value. It is boolean, so no average
- holiday: It will be the same to the same day. So get one value. It is boolean, so no average

Also calculate number of trips - count

So we have to calculate:

Count:
- [x] Count number of trips

Average:
- [x] tripduration
- [x] birth_year/age
- [x] hour

Copy value:
- [x] week_day
- [x] weekend
- [x] holiday


## Group by day

In [256]:
# number of trips in each day
daily_trips_number = trips['starttime'].groupby(pd.Grouper(freq='D')).count()
daily_trips_number

date
2020-02-16    3
2020-02-17    0
2020-02-18    2
2020-02-19    1
Freq: D, Name: starttime, dtype: int64

### Daily average

In [257]:
df = pd.DataFrame()
df

""


In [258]:
variable = 'tripduration'
df = trips[variable].groupby(pd.Grouper(freq='D')).sum()/daily_trips_number
df = pd.DataFrame(df, columns=[variable])
df

,tripduration
date,
2020-02-16,2408.0
2020-02-17,NaN
2020-02-18,707.5
2020-02-19,920.0


In [259]:
variable = 'hour'
variable_grouped = trips[variable].groupby(pd.Grouper(freq='D')).sum()/daily_trips_number
variable_grouped = pd.DataFrame(variable_grouped, columns=[variable])

In [260]:
pd.merge(df, variable_grouped, left_index=True, right_index=True)

,tripduration,hour
date,,
2020-02-16,2408.0,15.666667
2020-02-17,NaN,NaN
2020-02-18,707.5,9.000000
2020-02-19,920.0,17.000000


#### Birth year

Calculate age

In [261]:
trips['birth_year']

date
2020-02-16    2050.0
2020-02-16    1900.0
2020-02-16    2001.0
2020-02-18    1994.0
2020-02-18    1988.0
2020-02-19       NaN
Name: birth_year, dtype: float64

In [262]:
trips['birth_year'].apply(lambda x: pd.to_datetime(x, format='%Y'))

date
2020-02-16   2050-01-01
2020-02-16   1900-01-01
2020-02-16   2001-01-01
2020-02-18   1994-01-01
2020-02-18   1988-01-01
2020-02-19          NaT
Name: birth_year, dtype: datetime64[ns]

In [263]:
trips['birth_year'] = pd.to_datetime(trips['birth_year'], errors='coerce', format='%Y')

In [264]:
pd.options.mode.chained_assignment = None  # default='warn'
trips['age'] = trips["starttime"].dt.year - trips["birth_year"].dt.year
trips['age'][(trips['age'] < 0) | (trips['age'] > 100) ] = None

In [265]:
trips

,Index,trip_id,tripduration,start_station_name,starttime,end_station_name,stoptime,birth_year,initial_station_latitude,initial_station_longitude,final_station_latitude,final_station_longitude,per_day,hour,week_day,weekend,holiday,age
date,,,,,,,,,,,,,,,,,,
2020-02-16,1553109,6517937BikeSampa,724,6 - Rua Manduri,2020-02-16 23:52:18,223 -Fradique Coutinho,2020-02-17 00:04:22,2050-01-01,NaN,NaN,NaN,NaN,2020-02-16,23,6,True,False,NaN
2020-02-16,1553110,6517938BikeSampa,4332,204- Consolação,2020-02-16 23:52:49,204- Consolação,2020-02-17 01:05:01,1900-01-01,NaN,NaN,NaN,NaN,2020-02-16,23,6,True,False,NaN
2020-02-16,1553111,6517939BikeSampa,2168,95 - Metrô Brigadeiro,2020-02-16 01:53:35,153 - Monsenhor Passaláqua,2020-02-17 00:29:43,2001-01-01,NaN,NaN,NaN,NaN,2020-02-16,1,6,True,False,19.0
2020-02-18,1567892,6540675BikeSampa,787,12 - Largo de Batata II,2020-02-18 09:04:21,17 - Rua José Gonçalves de Oliveira,2020-02-18 09:17:28,1994-01-01,NaN,NaN,NaN,NaN,2020-02-18,9,1,False,True,26.0
2020-02-18,1567893,6540678BikeSampa,628,347 - Metrô Brooklin,2020-02-18 09:04:34,345 - R. Henrique FIsher,2020-02-18 09:15:02,1988-01-01,NaN,NaN,NaN,NaN,2020-02-18,9,1,False,True,32.0
2020-02-19,1584654,6564502BikeSampa,920,154 - R. Condessa de São Joaquim,2020-02-19 17:09:46,167 - Praca Roosevelt,2020-02-19 17:25:06,NaT,NaN,NaN,NaN,NaN,2020-02-19,17,2,False,False,NaN


In [266]:
# number of trips in each day
daily_trips_number = trips['age'].groupby(pd.Grouper(freq='D')).count()
daily_trips_number

date
2020-02-16    1
2020-02-17    0
2020-02-18    2
2020-02-19    0
Freq: D, Name: age, dtype: int64

In [267]:
# In order to find the daily average, I need to consider only non-null values

def find_daily_average_old(variable):
    daily_trips_number = trips[variable].groupby(pd.Grouper(freq='D')).count()
    df = trips[variable].groupby(pd.Grouper(freq='D')).sum()/daily_trips_number
    df = pd.DataFrame(df, columns=[variable])
    return df

def find_daily_average(variable):
    df = trips[variable].groupby(pd.Grouper(freq='D')).mean()
    df = pd.DataFrame(df, columns=[variable])
    return df

In [268]:
trips['age']

date
2020-02-16     NaN
2020-02-16     NaN
2020-02-16    19.0
2020-02-18    26.0
2020-02-18    32.0
2020-02-19     NaN
Name: age, dtype: float64

In [269]:
variable = 'age'
find_daily_average(variable)

,age
date,
2020-02-16,19.0
2020-02-17,NaN
2020-02-18,29.0
2020-02-19,NaN


In [270]:
# Some variables are dependent only on the day, so they will be same to every trip on the same day

def copy_daily_value(variable):
    df = trips[variable].groupby(pd.Grouper(freq='D')).first()
    df = pd.DataFrame(df, columns=[variable])
    return df

In [271]:
variable = 'weekend'
copy_daily_value(variable)

,weekend
date,
2020-02-16,1.0
2020-02-17,NaN
2020-02-18,0.0
2020-02-19,0.0


In [272]:
trips

,Index,trip_id,tripduration,start_station_name,starttime,end_station_name,stoptime,birth_year,initial_station_latitude,initial_station_longitude,final_station_latitude,final_station_longitude,per_day,hour,week_day,weekend,holiday,age
date,,,,,,,,,,,,,,,,,,
2020-02-16,1553109,6517937BikeSampa,724,6 - Rua Manduri,2020-02-16 23:52:18,223 -Fradique Coutinho,2020-02-17 00:04:22,2050-01-01,NaN,NaN,NaN,NaN,2020-02-16,23,6,True,False,NaN
2020-02-16,1553110,6517938BikeSampa,4332,204- Consolação,2020-02-16 23:52:49,204- Consolação,2020-02-17 01:05:01,1900-01-01,NaN,NaN,NaN,NaN,2020-02-16,23,6,True,False,NaN
2020-02-16,1553111,6517939BikeSampa,2168,95 - Metrô Brigadeiro,2020-02-16 01:53:35,153 - Monsenhor Passaláqua,2020-02-17 00:29:43,2001-01-01,NaN,NaN,NaN,NaN,2020-02-16,1,6,True,False,19.0
2020-02-18,1567892,6540675BikeSampa,787,12 - Largo de Batata II,2020-02-18 09:04:21,17 - Rua José Gonçalves de Oliveira,2020-02-18 09:17:28,1994-01-01,NaN,NaN,NaN,NaN,2020-02-18,9,1,False,True,26.0
2020-02-18,1567893,6540678BikeSampa,628,347 - Metrô Brooklin,2020-02-18 09:04:34,345 - R. Henrique FIsher,2020-02-18 09:15:02,1988-01-01,NaN,NaN,NaN,NaN,2020-02-18,9,1,False,True,32.0
2020-02-19,1584654,6564502BikeSampa,920,154 - R. Condessa de São Joaquim,2020-02-19 17:09:46,167 - Praca Roosevelt,2020-02-19 17:25:06,NaT,NaN,NaN,NaN,NaN,2020-02-19,17,2,False,False,NaN


In [273]:
variable = 'holiday'
trips[variable].groupby(pd.Grouper(freq='D')).first()

date
2020-02-16    0.0
2020-02-17    NaN
2020-02-18    1.0
2020-02-19    0.0
Freq: D, Name: holiday, dtype: float64

In [274]:
variable = 'week_day'
trips[variable].groupby(pd.Grouper(freq='D')).first()

date
2020-02-16    6.0
2020-02-17    NaN
2020-02-18    1.0
2020-02-19    2.0
Freq: D, Name: week_day, dtype: float64

## Join all frames

In [275]:
def find_daily_average(df, variable):
        '''
        In order to find the daily average, you need to consider only non-null values
        '''
        grouped_df = df[variable].groupby(pd.Grouper(freq='D')).mean()
        grouped_df = pd.DataFrame(grouped_df, columns=[variable])
        return grouped_df

def first_daily_value(df, variable):
    '''
        Some variables are dependent only on the day,
        so they will be same for every trip on the same day
    '''
    grouped_df = df[variable].groupby(pd.Grouper(freq='D')).first()
    grouped_df = pd.DataFrame(grouped_df, columns=[variable])
    return grouped_df

def find_number_daily_occurrences(df):
    '''
        Returns data frame with number of trips in each day
    '''
    variable = 'starttime'
    daily_trips_number = trips[variable].groupby(pd.Grouper(freq='D')).count()
    df = pd.DataFrame(daily_trips_number)
    df = df.rename(columns={variable: 'number_of_trips'})
    return df


In [276]:
df = find_number_daily_occurrences(trips)

for variable in ['tripduration', 'hour', 'age']:
    variable_df = find_daily_average(trips, variable)
    df = pd.merge(df, variable_df, left_index=True, right_index=True)

for variable in ['week_day', 'weekend', 'holiday']:
    variable_df = first_daily_value(trips, variable)
    df = pd.merge(df, variable_df, left_index=True, right_index=True)

df

,number_of_trips,tripduration,hour,age,week_day,weekend,holiday
date,,,,,,,
2020-02-16,3,2408.0,15.666667,19.0,6.0,1.0,0.0
2020-02-17,0,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-18,2,707.5,9.000000,29.0,1.0,0.0,1.0
2020-02-19,1,920.0,17.000000,NaN,2.0,0.0,0.0
